In [ ]:
# import some packages
import numpy as np
import matplotlib.pyplot as plt

# Path to images
im_pth = "../data/images/"

img_list = [
    'M0616C002000.jpg',
    'M0630C049000.jpg',
    'M0633C028000.jpg',
    'M0630C042000.jpg',
    'M0571C038000.jpg',
    'M0631C027000.jpg',
    'M0714C026000.jpg',
    'M0640C051000.jpg'
]

non_defective = [
    'M0377C017010.jpg', 
    'Bent Cure 0Pa TestName_20201222_165250.586_C034.jpg'
]

In [ ]:
def plot_imgs(img_array, img_labels):
    
    #Plot OK
    n = len(img_array)
    # Work out the number of rows
    n_rows = int((n / 2) if (n % 2 == 0) else ((n + 1) / 2))

    fig, axs = plt.subplots(nrows=n_rows, ncols=2, figsize=(8,8))

    if n_rows == 1:
        for i in range(n):
            axs[i].set_title(img_labels[i])
            axs[i].imshow(img_array[i])

    else:
        k=0
        for i in range(n_rows):
            for j in range(2):
                axs[i][j].set_title(img_labels[k])
                axs[i][j].imshow(img_array[k])
                k += 1

In [ ]:
# Load files
img_array_ok = [plt.imread(f'{im_pth}'+i) for i in non_defective]
img_array_defected = [plt.imread(f'{im_pth}'+i) for i in img_list]

#Plot OK
plot_imgs(img_array_ok, non_defective)

In [ ]:
from scipy.signal import convolve2d as sig_convolve2d
from scipy.ndimage import convolve1d as nd_convolve1d
from scipy.signal.windows import gaussian

class Kernel:
    """
    Creates and applies a filter to an input image.
    """
    
    def __init__(self, dim=2, kernel='gaussian', **kwargs):
        """
        
        :param dim:
        :param kernel: Type of kernel to create options include:
            'gaussian': gaussian kernel
            'custom': custom specified kernel
        :return:
        """
        
        self.dim = dim
        self.missing_list = []
        self.kernel_params = kwargs
        
        # Create a gaussian kernel
        if kernel=='gaussian':
            # Check all parameters are passed
            self.required_params = ['convolve_mode', 'boundary', 'size', 'std']
            self.check_required_params()
            
            self.kernel_type = kernel
            self.kernel_val = self.generate_gaussian_kernel() 
        
        # pass a custom kernel
        elif kernel == 'custom':
            # Check all parameters are passed
            self.required_params = ['convolve_mode', 'boundary', 'custom_kernel']
            self.check_required_params()
            
            self.kernel_type = kernel
            self.kernel_params = kwargs
            self.kernel_val = self.kernel_params['custom_kernel'] 
            
        else:
            raise ValueError('Kernel type not recognised. Allowable values: gaussian.')
           
        # Check all required parameters are there:
        if len(self.missing_list) > 0:
            raise ValueError(f'Missing required parameters: {self.missing_list}')
            
    def __lshift__(self, in_img):
        """
        Applies a kernel to an image and returns that image with the kernel applied. 
        :param in_img:
        :return:  
        """
        return self.apply_filter(in_img)
    
    def check_required_params(self):
        """
        Checks that all the required kwargs have been passed
        """
        
        for param in self.required_params:
            if param not in self.kernel_params.keys():
                missing_list.append(param)
            else:
                pass
        
    def generate_gaussian_kernel(self):
        """
        Generates a gaussian kernel
        
        self.key
        
        """
        #Extract size and sigma
        size = self.kernel_params['size']
        std = self.kernel_params['std']
        
        # Create 1d kernel
        gaussian_1d = gaussian(size, std, sym=True)
                  
        if self.dim == 1:
            return gaussian_1d / gaussian_1d.sum()
        
        elif self.dim == 2:
            #Create 2d filter and normalise 
            gaussian_2d = np.outer(gaussian_1d, gaussian_1d)
            return gaussian_2d / gaussian_2d.sum()
                         
    
    def apply_filter(self, in_img):
        """
        Wrapper for the scipy.signal.convolve2d method:
        https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve2d.html#scipy.signal.convolve2d
        Wrapper for the scipy.signal.convolve1d method:
        https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.convolve1d.html
        
        :param in_img: The array to apply the filter to
        :return: array with the filter applied
        """          
        
        if self.dim == 1:
            return nd_convolve1d(in_img, self.kernel_val, mode='wrap')
        
        elif self.dim == 2:
            #Create 2d filter and normalise 
            return sig_convolve2d(in_img, self.kernel_val, 
                                  mode = self.kernel_params['convolve_mode'],
                                  boundary = self.kernel_params['boundary'])
        
        
        

In [ ]:
import cv2
img = cv2.cvtColor(cv2.imread(im_pth+non_defective[0]), cv2.COLOR_BGR2GRAY) 

In [ ]:
plt.imshow(img)

In [ ]:
gaussian_filter2d = Kernel(dim=2, kernel='gaussian',
                        convolve_mode='full',boundary='wrap', size=10, std=2)

plt.imshow(gaussian_filter2d << img)

In [ ]:
gaussian_filter1d = Kernel(dim=1, kernel='gaussian',
                        convolve_mode='full',boundary='wrap', size=10, std=2)
plt.imshow(gaussian_filter1d << img)

In [ ]:
sobel = np.array([[1,0,-1],[2,0,-2],[1,0,-1]])

custom_filter2d = Kernel(dim=2, kernel='custom',
                        convolve_mode='full',boundary='wrap', 
                        custom_kernel=sobel)

plt.imshow(custom_filter2d << img)

In [ ]:
sobel = np.array([[1,0,-1],[2,0,-2],[1,0,-1]])

custom_filter2d = Kernel(dim=2, kernel='custom',
                        convolve_mode='full',boundary='wrap', 
                        custom_kernel=sobel.T)

plt.imshow(custom_filter2d << img)